In [1]:
import pyspark
from datetime import datetime as dt

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')  

In [2]:
# RDD de la forma [(padron, materia, nota, fecha), ...]
# Otro RDD con forma [(padron, nombre), ...]

# A) Alumnos que hayan aprobado al menos una materia en los ultimos dos años
# B) RDD con el promedio de notas de cada alumno por padron
# C) Nombre y padron del alumno con el mejor promedio

In [3]:
notas = [(98446, '75.06', 95, '12-10-2019'),
         (71532, '64.01', 60, '03-12-2009'),
         (83456, '76.02', 40, '02-20-2019'),
         (123634, '96.20', 20, '10-10-2000'),
         (100231, '66.02', 30, '02-13-2007'),
         (54632, '75.06', 60, '08-02-2014'),
         (96545, '90.02', 70, '09-12-2010'),
         (98446, '64.01', 80, '12-21-2003'),
         (54256, '77.70', 10, '02-12-2011'),
         (98446, '75.08', 65, '12-10-2019'),
         (71532, '64.03', 60, '03-12-2009'),
         (83456, '76.04', 40, '02-20-2019'),
         (123634, '96.25', 10, '10-10-2000'),
         (100231, '66.04', 100, '02-13-2007'),
         (54632, '75.02', 60, '08-02-2014'),
         (96545, '90.06', 60, '09-12-2010'),
         (98446, '64.12', 70, '12-29-2003'),
         (54256, '77.70', 40, '02-20-2011'),
         (101020, '75.06', 60, '11-12-2019')
        ]

date_format = '%m-%d-%Y'

notas_rdd = sc.parallelize(notas).map(lambda x: (x[0], x[1], x[2], dt.strptime(x[3], date_format) ))

In [4]:
cutoff_date = dt.strptime('11-13-2017', date_format)

aprobados = notas_rdd.filter(lambda x: x[2] >= 60 and x[3] >= cutoff_date).keys().distinct()
aprobados.take(5)

[101020, 98446]

In [5]:
promedios = sc.parallelize(notas).map(lambda x: (x[0], (x[2], 1))).reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1])).map(lambda x: (x[0], x[1][0]/x[1][1]))
promedios.take(5)

[(83456, 40.0), (54632, 60.0), (101020, 60.0), (71532, 60.0), (54256, 25.0)]

In [6]:
nombres = [
    (98446, 'Berrotaran Pablo'),
    (101020, 'Pinera, Agustin'),
    (54256, 'Gomez, Jorge'),
    (96545, 'Fernandez, Fernando'),
    (54632, 'Jamiroquai, Pepito'),
    (100231, 'Redis, Camila'),
    (123634, 'Postgres, Rodolfo'),
    (83456, 'Mongo, Dongo'),
    (71532, 'Kibana, Mutimbi'),
]

nombres_rdd = sc.parallelize(nombres).map(lambda x: (x[0], (x[1])))

top_promedio = sc.parallelize(promedios.top(1, key=lambda x: x[1])).map(lambda x: (x[0],(x[1]))).join(nombres_rdd).map(lambda x: (x[0], x[1][1], x[1][0]))
top_promedio.take(5)

[(98446, 'Berrotaran Pablo', 77.5)]